In [2]:
import os

# Define the path to the local Datasets folder
dataset_path = 'Datasets'

# Verify if the path exists
if os.path.exists(dataset_path):
    print(f"Dataset path found at: {dataset_path}")
else:
    print(f"Warning: '{dataset_path}' not found. Please ensure the 'Datasets' folder exists in the current directory.")


Dataset path found at: Datasets


In [4]:
import pandas as pd
import glob
import re

# Find all CSV files recursively
all_files = glob.glob(os.path.join(dataset_path, '**', '*.csv'), recursive=True)

df_list = []

for filename in all_files:
    # Read the CSV
    temp_df = pd.read_csv(filename)

    # Extract the week name from the filename (e.g., 'week_1.csv' -> 'week_1')
    week_name = os.path.splitext(os.path.basename(filename))[0]

    # Add a column to identify the week
    temp_df['week'] = week_name

    df_list.append(temp_df)

# Concatenate all dataframes into one
if df_list:
    df = pd.concat(df_list, ignore_index=True)

    # Display the first few rows to verify the 'week' column
    display(df.head())

    # Verify the unique weeks loaded
    print("\nWeeks loaded:")
    print(sorted(df['week'].unique(), key=lambda x: int(re.search(r'\d+', x).group()) if re.search(r'\d+', x) else 0))
else:
    print("No CSV files found.")

,account_id,player_slot,team_number,team_slot,hero_id,hero_variant,item_0,item_1,item_2,item_3,...,purchase_time,first_purchase_time,item_win,item_usage,purchase_ward_observer,purchase_ward_sentry,purchase_tpscroll,actions_per_min,life_state_dead,cosmetics
0,1195957275,2,0,2,81,2,63,112,151,154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1195957275,1,0,1,74,3,29,108,65,279,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1195957275,3,0,3,102,1,50,939,137,4205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1195957275,131,1,3,105,2,259,180,102,108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1195957275,3,0,3,62,2,226,202,98,108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Weeks loaded:
['week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'week_9', 'week_10', 'week_11', 'week_12']


In [5]:
# Check for column consistency across all loaded DataFrames
if df_list:
    # Use the first DataFrame's columns as the reference
    reference_cols = list(df_list[0].columns)
    reference_set = set(reference_cols)

    mismatches = []

    for i, d in enumerate(df_list):
        current_cols_set = set(d.columns)
        if current_cols_set != reference_set:
            # Identify the difference
            missing = reference_set - current_cols_set
            extra = current_cols_set - reference_set

            # Get the week name for reporting
            week_val = d['week'].iloc[0] if 'week' in d.columns and not d.empty else f"DataFrame {i}"

            msg = f"Week {week_val}:"
            if missing:
                msg += f"\n  Missing columns (present in reference): {missing}"
            if extra:
                msg += f"\n  Extra columns (not in reference): {extra}"
            mismatches.append(msg)

    if not mismatches:
        print(f"Success: All {len(df_list)} CSV files have the same columns.")
        print(f"Columns found: {reference_cols}")
    else:
        print("Mismatch found in columns:")
        for m in mismatches:
            print(m)
else:
    print("df_list is empty. No data to check.")

Mismatch found in columns:
Week week_7:
  Extra columns (not in reference): {'obs_left_log', 'pings', 'damage_inflictor', 'roshans_killed', 'lane_efficiency_pct', 'observer_kills', 'lane', 'item_uses', 'neutral_kills', 'rune_pickups', 'gold_reasons', 'purchase_ward_observer', 'xp_reasons', 'killed', 'gold_t', 'teamfight_participation', 'tower_kills', 'life_state', 'lane_efficiency', 'firstblood_claimed', 'runes', 'purchase_time', 'kills_log', 'camps_stacked', 'max_hero_hit', 'item_win', 'xp_t', 'randomed', 'multi_kills', 'observers_placed', 'sentry_uses', 'lh_t', 'sentry_kills', 'first_purchase_time', 'times', 'sen_log', 'runes_log', 'actions', 'hero_kills', 'ability_targets', 'cosmetics', 'sen_placed', 'pred_vict', 'damage_inflictor_received', 'purchase_log', 'necronomicon_kills', 'creeps_stacked', 'dn_t', 'obs_log', 'life_state_dead', 'obs', 'purchase_tpscroll', 'healing', 'kill_streaks', 'observer_uses', 'hero_hits', 'obs_placed', 'connection_log', 'buyback_log', 'is_roaming', 'cour

In [6]:
# Re-combine the DataFrames keeping only the common columns
if 'df_list' in locals() and df_list:
    # join='inner' keeps only the intersection of columns from all DataFrames
    df = pd.concat(df_list, join='inner', ignore_index=True)

    print("Extra columns removed. Kept only columns common to all weeks.")
    print(f"Updated DataFrame shape: {df.shape}")
    print(f"Columns remaining: {len(df.columns)}")
else:
    print("df_list is not defined. Please run the data loading cell first.")

Extra columns removed. Kept only columns common to all weeks.
Updated DataFrame shape: (103, 66)
Columns remaining: 66


In [7]:
# Convert 'start_time' from Unix timestamp to readable datetime
if 'start_time' in df.columns:
    # Convert using unit='s' for seconds
    df['start_time_readable'] = pd.to_datetime(df['start_time'], unit='s')

    print("Converted 'start_time' to 'start_time_readable'.")
    display(df[['start_time', 'start_time_readable']].head())
else:
    print("Column 'start_time' not found.")
    # Check for other potential time-related columns
    time_cols = [col for col in df.columns if 'time' in col.lower()]
    if time_cols:
        print(f"Other potential time columns found: {time_cols}")

Converted 'start_time' to 'start_time_readable'.


,start_time,start_time_readable
0,1769144328,2026-01-23 04:58:48
1,1769141698,2026-01-23 04:14:58
2,1769057458,2026-01-22 04:50:58
3,1769054208,2026-01-22 03:56:48
4,1768973566,2026-01-21 05:32:46


In [8]:
# Save the combined DataFrame to a CSV file
output_csv_path = 'combined_player_data.csv'

if 'df' in locals() and not df.empty:
    df.to_csv(output_csv_path, index=False)
    print(f"Successfully saved the combined dataset to: {output_csv_path}")
    print(f"File size: {os.path.getsize(output_csv_path) / (1024*1024):.2f} MB")
else:
    print("DataFrame 'df' is missing or empty. Please run the previous steps first.")


Successfully saved the combined dataset to: combined_player_data.csv
File size: 0.09 MB


In [9]:
# Check for missing values in the combined DataFrame
if 'df' in locals() and not df.empty:
    missing_count = df.isnull().sum()
    missing_percentage = (df.isnull().sum() / len(df)) * 100

    missing_data = pd.DataFrame({'Missing Count': missing_count, 'Missing Percentage': missing_percentage})

    # Filter to show only columns with missing values
    missing_data = missing_data[missing_data['Missing Count'] > 0].sort_values(by='Missing Count', ascending=False)

    if not missing_data.empty:
        print("Columns with missing values:")
        display(missing_data)
    else:
        print("Great! No missing values found in the combined dataset.")
else:
    print("DataFrame 'df' is not available. Please run the data loading steps first.")

Columns with missing values:


,Missing Count,Missing Percentage
name,103,100.000000
last_login,103,100.000000
region,59,57.281553
permanent_buffs,59,57.281553
party_id,59,57.281553
party_size,59,57.281553
kills_per_min,4,3.883495


In [10]:
# Drop 'name' and 'last_login' columns
cols_to_drop = ['name', 'last_login']

# Drop them if they exist
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

print(f"Dropped columns: {cols_to_drop}")
print(f"Remaining columns: {len(df.columns)}")
display(df.head())

Dropped columns: ['name', 'last_login']
Remaining columns: 65


,account_id,player_slot,team_number,team_slot,hero_id,hero_variant,item_0,item_1,item_2,item_3,...,abandons,benchmarks,match_id,region,party_id,permanent_buffs,party_size,start_time_iso,week,start_time_readable
0,1195957275,2,0,2,81,2,63,112,151,154,...,0,"{'gold_per_min': {'raw': 659, 'pct': 0.8960396...",8660412202,NaN,NaN,NaN,NaN,2026-01-23 04:58:48,week_10,2026-01-23 04:58:48
1,1195957275,1,0,1,74,3,29,108,65,279,...,0,"{'gold_per_min': {'raw': 362, 'pct': 0.2110009...",8660387354,15.0,NaN,NaN,NaN,2026-01-23 04:14:58,week_10,2026-01-23 04:14:58
2,1195957275,3,0,3,102,1,50,939,137,4205,...,0,"{'gold_per_min': {'raw': 331, 'pct': 0.3785488...",8659172466,NaN,NaN,NaN,NaN,2026-01-22 04:50:58,week_10,2026-01-22 04:50:58
3,1195957275,131,1,3,105,2,259,180,102,108,...,0,"{'gold_per_min': {'raw': 442, 'pct': 0.6259204...",8659142743,15.0,NaN,NaN,NaN,2026-01-22 03:56:48,week_10,2026-01-22 03:56:48
4,1195957275,3,0,3,62,2,226,202,98,108,...,0,"{'gold_per_min': {'raw': 493, 'pct': 0.6225596...",8657961379,15.0,NaN,NaN,NaN,2026-01-21 05:32:46,week_10,2026-01-21 05:32:46


In [11]:
# Drop additional columns: 'party_id', 'party_size', 'permanent_buffs', 'region'
more_cols_to_drop = ['party_id', 'party_size', 'permanent_buffs', 'region']

# Drop them if they exist
df.drop(columns=more_cols_to_drop, inplace=True, errors='ignore')

print(f"Dropped columns: {more_cols_to_drop}")
print(f"Remaining columns: {len(df.columns)}")
display(df.head())

Dropped columns: ['party_id', 'party_size', 'permanent_buffs', 'region']
Remaining columns: 61


,account_id,player_slot,team_number,team_slot,hero_id,hero_variant,item_0,item_1,item_2,item_3,...,total_gold,total_xp,kills_per_min,kda,abandons,benchmarks,match_id,start_time_iso,week,start_time_readable
0,1195957275,2,0,2,81,2,63,112,151,154,...,25624,46348,0.385769,2.73,0,"{'gold_per_min': {'raw': 659, 'pct': 0.8960396...",8660412202,2026-01-23 04:58:48,week_10,2026-01-23 04:58:48
1,1195957275,1,0,1,74,3,29,108,65,279,...,14009,13970,0.206718,1.19,0,"{'gold_per_min': {'raw': 362, 'pct': 0.2110009...",8660387354,2026-01-23 04:14:58,week_10,2026-01-23 04:14:58
2,1195957275,3,0,3,102,1,50,939,137,4205,...,13957,21083,0.023715,1.73,0,"{'gold_per_min': {'raw': 331, 'pct': 0.3785488...",8659172466,2026-01-22 04:50:58,week_10,2026-01-22 04:50:58
3,1195957275,131,1,3,105,2,259,180,102,108,...,20457,25640,0.302485,1.33,0,"{'gold_per_min': {'raw': 442, 'pct': 0.6259204...",8659142743,2026-01-22 03:56:48,week_10,2026-01-22 03:56:48
4,1195957275,3,0,3,62,2,226,202,98,108,...,26934,34364,0.164735,1.24,0,"{'gold_per_min': {'raw': 493, 'pct': 0.6225596...",8657961379,2026-01-21 05:32:46,week_10,2026-01-21 05:32:46


In [12]:
# Drop rows where 'kills_per_min' is missing
if 'kills_per_min' in df.columns:
    initial_rows = len(df)
    df.dropna(subset=['kills_per_min'], inplace=True)
    dropped_rows = initial_rows - len(df)

    print(f"Dropped {dropped_rows} rows with missing 'kills_per_min'.")
    print(f"Current DataFrame shape: {df.shape}")
else:
    print("Column 'kills_per_min' not found in DataFrame.")

Dropped 4 rows with missing 'kills_per_min'.
Current DataFrame shape: (99, 61)


In [13]:
# Check for columns related to winning or match outcome
win_cols = [col for col in df.columns if 'win' in col.lower() or 'vict' in col.lower()]

if win_cols:
    print(f"Potential win/loss columns found: {win_cols}")

    for col in win_cols:
        print(f"\nValue counts for '{col}':")
        print(df[col].value_counts(dropna=False))
else:
    print("No explicit 'win' or 'victory' columns found. Checking for 'radiant_win' specifically...")
    if 'radiant_win' in df.columns:
        print("Found 'radiant_win'.")
        print(df['radiant_win'].value_counts(dropna=False))
    else:
        print("Warning: Could not find standard win/loss labels.")

Potential win/loss columns found: ['radiant_win', 'win']

Value counts for 'radiant_win':
radiant_win
True     51
False    48
Name: count, dtype: int64

Value counts for 'win':
win
1    51
0    48
Name: count, dtype: int64


In [14]:
import requests

# Fetch hero data from OpenDota API
url = "https://api.opendota.com/api/heroes"
response = requests.get(url)

if response.status_code == 200:
    heroes_data = response.json()
    # Create a mapping dictionary: hero_id -> localized_name
    hero_map = {hero['id']: hero['localized_name'] for hero in heroes_data}

    print(f"Successfully fetched {len(hero_map)} heroes.")

    # Map the hero_id to hero_name in the DataFrame
    if 'hero_id' in df.columns:
        df['hero_name'] = df['hero_id'].map(hero_map)
        print("Added 'hero_name' column to DataFrame.")
        display(df[['hero_id', 'hero_name']].head())
    else:
        print("Column 'hero_id' not found in DataFrame.")
else:
    print(f"Failed to fetch hero data. Status Code: {response.status_code}")

Successfully fetched 127 heroes.
Added 'hero_name' column to DataFrame.


,hero_id,hero_name
0,81,Chaos Knight
1,74,Invoker
2,102,Abaddon
3,105,Techies
4,62,Bounty Hunter


In [15]:
import requests
import numpy as np

# Fetch item constants from OpenDota API
url_items = "https://api.opendota.com/api/constants/items"
response_items = requests.get(url_items)

if response_items.status_code == 200:
    items_data = response_items.json()

    # Create a mapping dictionary: item_id -> dname (display name)
    # The API returns a dict where keys are internal names, values have 'id' and 'dname'
    item_map = {}
    for key, val in items_data.items():
        if 'id' in val and 'dname' in val:
            item_map[val['id']] = val['dname']

    print(f"Successfully fetched {len(item_map)} items.")

    # List of item columns to map
    item_cols = [
        'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5',
        'backpack_0', 'backpack_1', 'backpack_2',
        'item_neutral', 'item_neutral2'
    ]

    # Create new columns with item names
    new_cols_created = []
    for col in item_cols:
        if col in df.columns:
            new_col_name = f"{col}_name"
            # Ensure the column is numeric (handle potential string issues)
            # Use map with a fallback for unmapped IDs (e.g., 0 or NaNs)
            df[new_col_name] = df[col].map(item_map)
            new_cols_created.append(new_col_name)

    print(f"Mapped {len(new_cols_created)} item columns.")
    display(df[new_cols_created].head())
else:
    print(f"Failed to fetch item data. Status Code: {response_items.status_code}")

Successfully fetched 470 items.
Mapped 11 item columns.


,item_0_name,item_1_name,item_2_name,item_3_name,item_4_name,item_5_name,backpack_0_name,backpack_1_name,backpack_2_name,item_neutral_name,item_neutral2_name
0,Power Treads,Assault Cuirass,Armlet of Mordiggian,Sange and Yasha,Heart of Tarrasque,Magic Wand,Greater Healing Lotus,Quelling Blade,NaN,Duelist Gloves,Mystical
1,Boots of Speed,Aghanim's Scepter,Hand of Midas,Ring of Tarrasque,Tiara of Selemene,NaN,NaN,NaN,NaN,Essence Ring,Vampiric
2,Phase Boots,Harpoon,Radiance,Great Healing Lotus,Gem of True Sight,Magic Wand,NaN,NaN,NaN,Serrated Shiv,Alert
3,Kaya,Arcane Boots,Force Staff,Aghanim's Scepter,Aether Lens,Octarine Core,NaN,Observer Ward,NaN,Whisper of the Dread,Mystical
4,Lotus Orb,Dagon,Orchid Malevolence,Aghanim's Scepter,Aether Lens,Tranquil Boots,Magic Wand,NaN,NaN,Flayer's Bota,Quickened


In [16]:
print(f"Total columns: {len(df.columns)}")
print(df.columns.tolist())

Total columns: 73
['account_id', 'player_slot', 'team_number', 'team_slot', 'hero_id', 'hero_variant', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'backpack_0', 'backpack_1', 'backpack_2', 'item_neutral', 'item_neutral2', 'kills', 'deaths', 'assists', 'leaver_status', 'last_hits', 'denies', 'gold_per_min', 'xp_per_min', 'level', 'net_worth', 'aghanims_scepter', 'aghanims_shard', 'moonshard', 'hero_damage', 'tower_damage', 'hero_healing', 'gold', 'gold_spent', 'ability_upgrades_arr', 'personaname', 'rank_tier', 'computed_mmr', 'is_subscriber', 'radiant_win', 'start_time', 'duration', 'cluster', 'lobby_type', 'game_mode', 'is_contributor', 'patch', 'isRadiant', 'win', 'lose', 'total_gold', 'total_xp', 'kills_per_min', 'kda', 'abandons', 'benchmarks', 'match_id', 'start_time_iso', 'week', 'start_time_readable', 'hero_name', 'item_0_name', 'item_1_name', 'item_2_name', 'item_3_name', 'item_4_name', 'item_5_name', 'backpack_0_name', 'backpack_1_name', 'backpack_2_name', 'ite

In [17]:
cols_to_remove = [
    'account_id', 'personaname', 'is_subscriber', 'is_contributor',
    'hero_variant', 'start_time_iso', 'start_time_readable',
    'cluster', 'lobby_type', 'radiant_win', 'lose', 'week'
]

# Drop the columns if they exist
df.drop(columns=cols_to_remove, inplace=True, errors='ignore')

print(f"Dropped columns: {cols_to_remove}")
print(f"Remaining columns: {len(df.columns)}")
display(df.head())

Dropped columns: ['account_id', 'personaname', 'is_subscriber', 'is_contributor', 'hero_variant', 'start_time_iso', 'start_time_readable', 'cluster', 'lobby_type', 'radiant_win', 'lose', 'week']
Remaining columns: 61


,player_slot,team_number,team_slot,hero_id,item_0,item_1,item_2,item_3,item_4,item_5,...,item_1_name,item_2_name,item_3_name,item_4_name,item_5_name,backpack_0_name,backpack_1_name,backpack_2_name,item_neutral_name,item_neutral2_name
0,2,0,2,81,63,112,151,154,114,36,...,Assault Cuirass,Armlet of Mordiggian,Sange and Yasha,Heart of Tarrasque,Magic Wand,Greater Healing Lotus,Quelling Blade,NaN,Duelist Gloves,Mystical
1,1,0,1,74,29,108,65,279,1802,0,...,Aghanim's Scepter,Hand of Midas,Ring of Tarrasque,Tiara of Selemene,NaN,NaN,NaN,NaN,Essence Ring,Vampiric
2,3,0,3,102,50,939,137,4205,30,36,...,Harpoon,Radiance,Great Healing Lotus,Gem of True Sight,Magic Wand,NaN,NaN,NaN,Serrated Shiv,Alert
3,131,1,3,105,259,180,102,108,232,235,...,Arcane Boots,Force Staff,Aghanim's Scepter,Aether Lens,Octarine Core,NaN,Observer Ward,NaN,Whisper of the Dread,Mystical
4,3,0,3,62,226,202,98,108,232,214,...,Dagon,Orchid Malevolence,Aghanim's Scepter,Aether Lens,Tranquil Boots,Magic Wand,NaN,NaN,Flayer's Bota,Quickened


In [18]:
# Save the final cleaned DataFrame to CSV
if 'df' in locals() and not df.empty:
    df.to_csv(output_csv_path, index=False)
    print(f"Successfully saved the final dataset to: {output_csv_path}")
    print(f"File size: {os.path.getsize(output_csv_path) / (1024*1024):.2f} MB")
else:
    print("DataFrame 'df' is missing or empty.")

Successfully saved the final dataset to: combined_player_data.csv
File size: 0.09 MB
